In [14]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import time
from urllib.parse import urlencode
import webbrowser

In [15]:
CLIENT_ID = '7e5b7f9df7fc403cb0bb2f2daececb2a'
CLIENT_SECRET = '5c40be6905884180883e166c03b4df8c'
REDIRECT_URI = 'http://localhost:8888/callback'

In [16]:
SCOPE = 'user-read-playback-state user-modify-playback-state user-read-currently-playing'

In [17]:
params = urlencode({
    'client_id': CLIENT_ID,
    'response_type': 'code',
    'redirect_uri': REDIRECT_URI,
    'scope': SCOPE
})

In [12]:
auth_url = f"https://accounts.spotify.com/authorize?{params}"
webbrowser.open(auth_url)
print("Abra este link no navegador:", auth_url)

Abra este link no navegador: https://accounts.spotify.com/authorize?client_id=7e5b7f9df7fc403cb0bb2f2daececb2a&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback&scope=user-read-playback-state+user-modify-playback-state+user-read-currently-playing


In [18]:
CODE = "AQD3Rnv-jz2_2BPXlcWg43uqcVshk1z9CLw2St3J_e7UEG76ssbkW9E93XutjtIkPG9eeQlAsjWhSc6HooD9nuLVspEr53hMIXwiE1XcQdUnk4H2X1H7UxRVaiz4N0R5tuqjztoOE0Mp4o8HRwl5XTAoWWfLl7t1QwuDeJmJATMZULQVOVXqNYYnbFYGhh_y-gM8D3yhb5ddXUMPm76N8w752mAnPu0pExPRZH-3jyCF8O_r8mFhCGiEjMt40xWFtlkpC3aaUQEbTFMXpbaUF_n4AG4k"

In [19]:
import requests
import base64

In [20]:
auth_header = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode()).decode()

url = "https://accounts.spotify.com/api/token"
headers = {
    "Authorization": f"Basic {auth_header}",
    "Content-Type": "application/x-www-form-urlencoded"
}

In [21]:
data = {
    "grant_type": "authorization_code",
    "code": CODE,
    "redirect_uri": REDIRECT_URI
}

In [22]:
response = requests.post(url, headers=headers, data=data)
if response.status_code == 200:
    tokens = response.json()
    print("Access Token:", tokens['access_token'])
    print("Refresh Token:", tokens['refresh_token'])
else:
    print(f"Erro {response.status_code}: {response.text}")

Access Token: BQBWNcm_b0Mz-4hFuxGEAMIkJbDdJlxKqV81XRuNvIloEkqZAS9bCKeTB-SD8UAU-IZzBog6DxwsjULIQjrtYCSEivCYhL7luhV8ZrJRRtb0RudjGAK0Guv5AeEWVrGLFIU8FCTgzGwicucogJ9BjZlBN7pRFlks-VmWHTZxtNKL0IKOvrpJUGrK9E84vGrJl5qfERfreJO9bslOrJNX6ZwOB7HphlBOLHAonOQURL9FkQtGPz7paN0
Refresh Token: AQC2W9npAXYVNfiKt-K-0vLq7r25AaFF2heqZIiw3UPW0bA2GyCiQrez-ksR7JMvTsGOWoVYN7WhechvO55F-9F-ma8AsV8y_XfJW9oX-CaotBVTdpSfJFab_hQ5YefZR8U


In [23]:
access_token = tokens['access_token']

In [28]:
url_busca = "https://api.spotify.com/v1/search"
headers = {
    "Authorization": f"Bearer {access_token}"
}

In [29]:
nome_musica = "fã número 1"

params = {
    "q": nome_musica,
    "type": "track",
    "limit": 1
}

In [30]:
response_busca = requests.get(url_busca, headers=headers, params=params)
resultados = response_busca.json()

In [ ]:

if 'tracks' in resultados and resultados['tracks']['items']:
    track = resultados['tracks']['items'][0]
    track_uri = track['uri']
    print(f"Música encontrada: {track['name']} - URI: {track_uri}")

    url_play = "https://api.spotify.com/v1/me/player/play"
    headers_play = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    data_play = {
        "uris": [track_uri]
    }
    response_play = requests.put(url_play, headers=headers_play, json=data_play)

    if response_play.status_code == 204:
        print("Música iniciada com sucesso!")
    else:
        print(f"Erro ao iniciar música: {response_play.status_code} - {response_play.text}")
else:
    print("Nenhuma música encontrada com esse nome.")

Música encontrada: Fã - URI: spotify:track:3DtT5SMTOJ9CgzmYsHEF86
Música iniciada com sucesso!


In [ ]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    redirect_uri=REDIRECT_URI,
    scope=SCOPE,
    open_browser=True 
))

In [ ]:
devices = sp.devices()

In [ ]:
if not devices['devices']:
    print("⚠ Nenhum dispositivo ativo. Abra o app do Spotify e toque algo manualmente.")
else:
    device = devices['devices'][0]
    device_id = device['id']
    print(f"🎧 Dispositivo ativo: {device['name']}")

    sp.pause_playback(device_id=device_id)
    print("⏸ Música pausada.")
    time.sleep(2)

    sp.start_playback(device_id=device_id)
    print("▶ Música tocando.")